In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
def imregionalmax(image, kernel=None):
    """Find the regional max of the image. An approximation of MATLAB's
    imregionalmax function. Result only differs when surrounding pixels
    have the same value as the center.

    Parameters:
    - image: the input image
    - kernel: the size of the neiborhood region, default is 3x3, i.e.
              neighboring 8 pixels.
    Returns:
    - a bitmask image, where '1' indicates local maxima.
    Author:
    - Yu Fang
    References:
    - https://github.com/bhardwajvijay/Utils/blob/master/utils.cpp
    - https://stackoverflow.com/questions/5550290/find-local-maxima-in-grayscale-image-using-opencv
    """
    # dialate the image so that small values are replaced by local max
    local_max = cv2.dilate(image, kernel)
    # non-local max pixels (excluding pixel w/ constant 3x3 neighborhood)
    # will be replaced by local max, so the values will increase. remove them.
    # so the result is either local max or constant neighborhood.
    max_mask = image >= local_max
    # erode the image so that high values are replaced by local min
    local_min = cv2.erode(image, kernel)
    # only local min pixels and pixels w/ constant 3x3 neighborhood
    # will stay the same, otherwise pixels will be replaced by the local
    # min and become smaller. We only take non-local min, non-constant values.
    min_mask = image > local_min
    # boolean logic hack
    #   (local max || constant) && (!local min && !constant)
    # = local max && !local min && !constant
    # = local max && !constant
    return (max_mask & min_mask).astype(np.uint8)

In [4]:
def est_homography(src, dest):
    """ Compute the homography matrix from (x_src, y_src) to (x_dest, y_dest).
    Parameters:
    - src: (x,y) coordinates of N source pixels, where coordinates are row vectors,
           so the matrix has dimension Nx2 (N>=4).
    - dest: (x,y) coordinates of N destination pixels, where coordinates are row vectors,
            so the matrix has dimension Nx2 (N>=4).
    Returns:
    - the homography matrix such that H @ [x_src, y_src, 1].T = [x_dest, y_dest, 1].T
    Author:
    - Yu Fang
    """
    N = src.shape[0]
    if N != dest.shape[0]:
        raise ValueError("src and diff should have the same dimension")
    src_h = np.hstack((src, np.ones((N, 1))))
    A = np.array([np.block([[src_h[n], np.zeros(3), -dest[n, 0] * src_h[n]],
                            [np.zeros(3), src_h[n], -dest[n, 1] * src_h[n]]])
                  for n in range(N)]).reshape(2 * N, 9)
    [_, _, V] = np.linalg.svd(A)
    # take the right singular vector x corresponding to the least singular value
    # s.t. ||Ax - 0||^2 is minimized
    return V.T[:, 8].reshape(3, 3)

In [5]:
def apply_homography(H, src):
    """ Apply the homography H to src
    Parameters:
    - H: the 3x3 homography matrix
    - src: (x,y) coordinates of N source pixels, where coordinates are row vectors,
           so the matrix has dimension Nx2 (N>=4).
    Returns:
    - src: (x,y) coordinates of N destination pixels, where coordinates are row vectors,
           so the matrix has dimension Nx2 (N>=4).
    Author:
    - Yu Fang
    """
    src_h = np.hstack((src, np.ones((src.shape[0], 1))))
    dest =  src_h @ H.T
    return (dest / dest[:,[2]])[:,0:2]

In [6]:
def drawMatches(image1, kp1, image2, kp2, idx_pairs):
    """A wrapper around OpenCV's drawMatches.
    
    Parameters:
    - image1: the first image
    - kp1: *matrix indices* of the keypoints from image 1
           (Nx2 numpy array, where N is the number of keypoints)
    - image2: the second image
    - kp2: *matrix indices* of the keypoints from image 2 
           (Nx2 numpy array, where N is the number of keypoints)
    - idx_pairs: pairs of matching indices, e.g. if kp1[3] 
                 matches kp2[5], then idx_pairs=[[3,5],...]
                 (Kx2 numpy array, where K is the number of matches)
    Returns:
    - an image showing matching points
    Author:
    - Yu Fang
    """
    # note that the coordinates are reversed because the difference
    # between matrix indexing & coordinates.
    keypt1 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp1.tolist()]
    keypt2 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp2.tolist()]
    matches = [cv2.DMatch(pair[0], pair[1], 0) for pair in idx_pairs.tolist()]
    return cv2.drawMatches(image1, keypt1, image2, keypt2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [7]:
def amns(cornerImage, n_best):
    c_robust = 0.9
    local_max = imregionalmax(cornerImage)
    y_coords, x_coords = np.where(local_max==1)
    
    n_strong = y_coords.size
    r = np.zeros(n_strong)
    
    for i in range(n_strong):
        r[i] = float("Inf")
        
    for i in range(n_strong):
        y = y_coords[i]
        x = x_coords[i]
        
        mask = cornerImage[y,x] < c_robust*cornerImage
        
        ind = np.argwhere(mask) # (x,y)'s that suppress (x_i,y_i) in cornerImage
        dist = np.power(np.linalg.norm(ind-[y,x],2,axis=1),2)
        r[i] = np.inf if ind.size == 0 else np.amin(dist)
    
    indices = r.argsort()[-n_best:] # indices of the (x,y) locations of the best corners
    best = np.array(list(map(lambda i: (x_coords[i], y_coords[i]), indices)))
    return best

In [8]:
# create sift descriptors
def sift_descriptors(image, points):
    descriptors = np.zeros((points.shape[0],64))
    
    counter = 0
    
    for x_sub, y_sub in points:
        # pixel is in position (20,20)
        x_pix  = int(np.floor(x_sub))
        y_pix = int(np.floor(y_sub))
        
        leftborder = 0 if x_pix-20 < 0 else x_pix-20
        rightborder = image.shape[1]-1 if x_pix+19 >= image.shape[1] else x_pix+19
        topborder = 0 if y_pix-20 < 0 else y_pix-20
        bottomborder = image.shape[0]-1 if y_pix+19 >= image.shape[0] else y_pix+19
        neighborhood = image[topborder:(bottomborder+1), leftborder:(rightborder+1)]
        
        # need to pad neighborhood
        if neighborhood.shape != (40,40):
            
            diffLeft = np.absolute(leftborder-(x_pix-20))
            diffRight = np.absolute(rightborder-(x_pix+19))
            diffTop = np.absolute(topborder-(y_pix-20))
            diffBot = np.absolute(bottomborder-(y_pix+19))
            
            neighborhood = np.pad(neighborhood, ((diffTop, diffBot),(diffLeft, diffRight)), 'edge')
        # Gaussian filter
        blurNbrh = cv2.GaussianBlur(src=neighborhood, ksize=(5, 5), sigmaX=1)
        # resize
        blurNbrh = cv2.resize(blurNbrh, (8,8))
        # reshape the neighborhood
        blurNbrh = blurNbrh.reshape(64)
        # normalization
        blurNbrh = (blurNbrh - np.mean(blurNbrh))/np.std(blurNbrh)
        descriptors[counter] = blurNbrh
        counter = counter + 1
    return descriptors

In [9]:
# feature matching
def matching(descriptor1, descriptor2):
    threshold = 0.8
    pairs = []
    min_dist = 100 #check if there are any inliners/matches
    
    for i in range(descriptor1.shape[0]):
        dists = np.linalg.norm(descriptor1[i] - descriptor2, axis=1)
        first_index, second_index = np.argsort(dists)[:2] # indicies for the best and second best points in 2nd image
        min_dist = dists[first_index] if min_dist > dists[first_index] else min_dist
        
        if dists[first_index]/dists[second_index] < threshold:
            pairs.append(np.array([i,first_index]))
    return (min_dist < 4) , np.array(pairs)

In [10]:
# RANSAC
def ransac(pairs, points1, points2):
    N_max = 30
    paired_points1 = np.array(list(map(lambda i: points1[i], pairs[:,0])))
    paired_points2 = np.array(list(map(lambda i: points2[i], pairs[:,1])))
    
    set_of_inliners = []
    for k in range(N_max):
        random_indices = np.random.choice(pairs.shape[0], 4, replace=False)
        
        point11 = paired_points1[random_indices[0]]
        point12 = paired_points2[random_indices[0]]

        point21 = paired_points1[random_indices[1]]
        point22 = paired_points2[random_indices[1]]

        point31 = paired_points1[random_indices[2]]
        point32 = paired_points2[random_indices[2]]

        point41 = paired_points1[random_indices[3]]
        point42 = paired_points2[random_indices[3]]

        H = est_homography(np.array([point11, point21, point31, point41]), np.array([point12, point22, point32, point42]))


        Hp = apply_homography(H,paired_points1)
        inliners = []
        for i in range(pairs.shape[0]):
            if np.linalg.norm(Hp[i]-paired_points2[i], 2) < 4:
                inliners.append((paired_points1[i], paired_points2[i], pairs[i]))
        
        set_of_inliners.append(np.array(inliners))
        
        if len(inliners)/pairs.shape[0] > 0.9:
                break
    largest_inliner = set_of_inliners[-1]
    
    if len(set_of_inliners) == N_max:
        for inliner in set_of_inliners:
            if inliner.shape[0] > largest_inliner.shape[0]:
                largest_inliner = inliner
                
    return largest_inliner[:,2], est_homography(largest_inliner[:,0],largest_inliner[:,1])

In [11]:
# attach image1 to image2
def stitch_images(image1, image2):
    for y in range(image1.shape[0]):
        for x in range(image1.shape[1]):
            if image1[y,x] > image2[y,x]:
                image2[y,x] =image1[y,x]
    return image2

In [12]:
# blend images
def blending(image_list, H_list):
    # All H in H_list stitch to one base image, the base image is the last image in image_list
    # image list of size N, H_list size N-1
    # H1 transforms image1 to last image space
    
    size = image_list[-1].shape # default shape
    leftMax, rightMax, topMax, botMax = 0, image_list[-1].shape[1], 0, image_list[-1].shape[0]
    
    for i in range(image_list.shape[0]-1):
        corners = np.array([[0,0], [image_list[i].shape[1], 0], \
                            [0, image_list[i].shape[0]], [image_list[i].shape[1], image_list[i].shape[0]]])
        tcorners = apply_homography(H_list[i], corners) # image_i to base image
        print(tcorners)
        
        leftMax = int(np.floor(min(np.amin(tcorners[:,0]), leftMax)))
        topMax = int(np.floor(min(np.amin(tcorners[:,1]), topMax)))
        rightMax = int(np.floor(max(np.amax(tcorners[:,0]), rightMax)))
        botMax = int(np.floor(max(np.amax(tcorners[:,1]), botMax)))
    
    size = (rightMax-leftMax, botMax-topMax)
    translate = np.eye(3)
    translate[0,2] = np.absolute(leftMax)
    translate[1,2] = np.absolute(topMax)
    
    translation_matrices = np.matmul(translate, H_list) # the last H to go from image
    
    base = stitch_images(cv2.warpPerspective(image_list[-1], translate, size), np.zeros((size[1], size[0])))
    
    for i in range(image_list.shape[0]-1):
        transformed = cv2.warpPerspective(image_list[i], translation_matrices[i], size)
        base = stitch_images(transformed, base)
    
    return base
    

In [22]:
# importing images, test set
input1 = np.divide(cv2.imread("../images/set2/1.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
input2 = np.divide(cv2.imread("../images/set2/2.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
input3 = np.divide(cv2.imread("../images/set2/3.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')

In [23]:
# Harris corner detection
corner1 = cv2.cornerHarris(input1, blockSize=5, ksize=5, k=0.06)
corner2 = cv2.cornerHarris(input2, blockSize=5, ksize=5, k=0.06)
corner3 = cv2.cornerHarris(input3, blockSize=5, ksize=5, k=0.06)

In [24]:
hcorners1 = amns(corner1, 500)
hcorners2 = amns(corner2, 500)
hcorners3 = amns(corner3, 500)

In [1038]:
local_max = imregionalmax(corner1)
y_coords, x_coords = np.where(local_max==1)

In [25]:
descriptors1 = sift_descriptors(input1, hcorners1)
descriptors2 = sift_descriptors(input2, hcorners2)
descriptors3 = sift_descriptors(input3, hcorners3)

In [26]:
flag1, matches1 = matching(descriptors1,descriptors2)
flag2, matches2 = matching(descriptors3,descriptors2)
flag3, matches3 = matching(descriptors1,descriptors3)

In [27]:
l1, H1 = ransac(matches1, hcorners1, hcorners2)
l2, H2 = ransac(matches2, hcorners3, hcorners2)

In [28]:
sol = blending(np.array([input1, input3, input2]), np.array([H1,H2]))

[[-369.92055719 -177.96839219]
 [ 274.63311541   40.88777824]
 [-540.10315901  748.44678226]
 [ 158.60238461  758.58049192]]
[[ 294.18093465   32.54477557]
 [ 917.7874598  -180.69603803]
 [ 408.65581178  743.05577539]
 [1083.11135268  731.20045526]]


In [1056]:
image = drawMatches(cv2.imread("../images/set1/1.jpg", cv2.IMREAD_GRAYSCALE), hcorners1[:,[1,0]], cv2.imread("../images/set1/2.jpg", cv2.IMREAD_GRAYSCALE), hcorners2[:,[1,0]], l1)

In [29]:
cv2.imshow('../images/input/set1_panorama.jpg', sol)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
linput1 = np.divide(cv2.imread("../images/set3/1.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
linput2 = np.divide(cv2.imread("../images/set3/2.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
linput3 = np.divide(cv2.imread("../images/set3/3.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
linput4 = np.divide(cv2.imread("../images/set3/4.jpg", cv2.IMREAD_GRAYSCALE),255.0).astype('float32')
lcorner1 = cv2.cornerHarris(linput1, blockSize=5, ksize=5, k=0.06)
lcorner2 = cv2.cornerHarris(linput2, blockSize=5, ksize=5, k=0.06)
lcorner3 = cv2.cornerHarris(linput3, blockSize=5, ksize=5, k=0.06)
lcorner4 = cv2.cornerHarris(linput4, blockSize=5, ksize=5, k=0.06)

lhcorners1 = amns(lcorner1, 500)
lhcorners2 = amns(lcorner2, 500)
lhcorners3 = amns(lcorner3, 500)
lhcorners4 = amns(lcorner4, 500)

In [14]:
ldescriptors1 = sift_descriptors(linput1, lhcorners1)
ldescriptors2 = sift_descriptors(linput2, lhcorners2)
ldescriptors3 = sift_descriptors(linput3, lhcorners3)
ldescriptors4 = sift_descriptors(linput4, lhcorners4)

lflag1, lmatches1 = matching(ldescriptors1,ldescriptors4)
lflag2, lmatches2 = matching(ldescriptors2,ldescriptors4)
lflag3, lmatches3 = matching(ldescriptors3,ldescriptors4)
lflag4, lmatches4 = matching(ldescriptors2,ldescriptors1)

In [19]:
l1, lH1 = ransac(lmatches1, lhcorners1, lhcorners4)
l2, lH2 = ransac(lmatches2, lhcorners2, lhcorners4)
l3, lH3 = ransac(lmatches3, lhcorners3, lhcorners4)

In [18]:
lH1.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [20]:
sol = blending(np.array([linput1, linput2, linput3, linput4]), np.array([lH1,lH2,lH3]))

[[366.00000001 577.        ]
 [366.         577.        ]
 [366.         577.        ]
 [366.         577.        ]]
[[-355.46760046   87.04752388]
 [ 250.44945826  -65.55669601]
 [-299.72615135  772.85308256]
 [ 234.74721172  800.71631519]]
[[-259.92580145  -80.45642027]
 [ 408.01376684   26.17864592]
 [-262.81010091  893.01796194]
 [ 408.36261633  767.27660442]]


In [ ]:
image = drawMatches(cv2.imread("../images/set3/1.jpg", cv2.IMREAD_GRAYSCALE), lhcorners1[:,[1,0]], cv2.imread("../images/set3/2.jpg", cv2.IMREAD_GRAYSCALE), lhcorners2[:,[1,0]], lmatches1)

In [21]:
cv2.imshow('im', sol)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [897]:
image = drawMatches(cv2.imread("../images/set1/1.jpg", cv2.IMREAD_GRAYSCALE), hcorners1[:,[1,0]], cv2.imread("../images/set1/2.jpg", cv2.IMREAD_GRAYSCALE), hcorners2[:,[1,0]], matches1)

In [904]:
cv2.imwrite('../images/input/set2_12matches.jpg', image)

True

In [899]:
cv2.imwrite('../images/input/set1_12matches.jpg', image)

True

In [808]:
testH = np.arange(9).reshape((3,3))
testx = np.ones(2)
testx[0] = 2
apply_homography(testH, np.array([testx]))

array([[0.11111111, 0.55555556]])

In [809]:
testH

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [839]:
test = np.arange(27).reshape((3,3,3))
np.matmul(np.ones(9).reshape(3,3), test)

array([[[ 9., 12., 15.],
        [ 9., 12., 15.],
        [ 9., 12., 15.]],

       [[36., 39., 42.],
        [36., 39., 42.],
        [36., 39., 42.]],

       [[63., 66., 69.],
        [63., 66., 69.],
        [63., 66., 69.]]])